In [1]:
import os
import xml
import xml.etree.ElementTree as ET
import re
import pandas as pd
import numpy as np

path = 'data/source/2019_YearEnd_XML/' # Path to the unzipped folder from https://disclosurespreview.house.gov/
files = os.listdir(path) # Generates a list of all the .xml files in path
url = 'https://disclosurespreview.house.gov/lc/lcxmlrelease/2019/YY/' # Base URL to add a link to the actual filing for each row

############################################
## XML tree structure generated using ... ##
############################################
# for file in os.listdir(path):
#     t = ET.parse('data/source/2019_YearEnd_XML/' + file)
#     r = t.getroot()
    
#     def perf_func(elem, func, level=0):
#         func(elem,level)
#         for child in list(elem):
#             perf_func(child, func, level+1)

#     def print_level(elem,level):
#         taglev = str(level) + '-' + elem.tag
#         if taglev not in elements:
#             print(taglev)
#             elements.append(taglev)
            
#     perf_func(r, print_level)

l1 = ['filerType','organizationName','lobbyistPrefix','lobbyistFirstName','lobbyistMiddleName','lobbyistLastName','lobbyistSuffix','zipext','contactPrefix','contactName','senateRegID','houseRegID','reportYear','reportType','amendment','comments','signedDate','certifiedcontent','noContributions','pacs','contributions','lobbyistID','address2','contactPhone','state','country','address1','city','zip','contactEmail']
l2 = ['pac']
l3 = ['type','contributorName','payeeName','recipientName','amount','date','name']

data = [] # Empty list for Pandas dataframe later

for file in files: # For every .xml file in the list of files ...
    
    tree = ET.parse(path + file) # ... Establish the XML tree structure ... 
    root = tree.getroot() # ... and get the root element ...
    fields = [] # This is an empty list which will be filled by XML tags, which will become columns in our data frame
    
    for contribution in root.iter('contribution'): # For loop instantiated every time there is an individual contribution
        row = {} # Empty dictionary that represents a row and will later be added to the 'data' list for the dataframe
        
        row['url'] = url + file # Adds key, value pair to the row dictionary, value is a URL to the original data made up of a base URL and the specifc file

        for i in l1: # Loop through the first level of XML tags
            fields.append(root.find(i)) # root.find() returns an XML element, add it to the fields list

        for i in l2: # Loop through the second level of XML tags, just 'pac'
            if root.find('pacs').find(i) is not None: # this basically looks for a 'pac' tag in 'pacs'
                fields.append(root.find('pacs').find(i).find('name')) # append the name of the pac to the fields list

        for i in l3: # Loop through the third level of XML tags, the individual contributions level ... 
            fields.append(contribution.find(i)) # ... append each contribution tag element to the 'fields' list

        for field in fields: # Loop through all the fields we just added to the fields list ...
            if field is not None: # ... if it's not None ... 
                row[field.tag] = field.text # ... create a key, value pair in the row dictionary where the XML element tag is the key and the value is the text in that tag.
                
        for k,v in row.items(): # For each key, value pair of every item in the rows dictionary ...
            if '\n' in v: # ... if the value has a carriage return ...
                newstr = re.sub(r'\s*\n+\s*', '', v) # A variable that finds any carriage return surrounded by zero or more spaces and replaces it with nothing. 
                row[k] = newstr # ... set the key to the new clean string without carriage returns
            
        data.append(row) # Add the row of data to our data called list.

df = pd.DataFrame(data) # Make a dataframe of the list of rows called 'data'
df = df[['contributorName', 'payeeName', 'recipientName', 'amount', 'date', 'name', 'url', 'filerType', 'organizationName', 'lobbyistPrefix', 'lobbyistFirstName', 'lobbyistMiddleName', 'lobbyistLastName', 'lobbyistSuffix', 'zipext', 'contactName', 'senateRegID', 'houseRegID', 'reportYear', 'reportType', 'amendment', 'comments', 'signedDate', 'certifiedcontent', 'noContributions', 'pacs', 'contributions', 'lobbyistID', 'address2', 'type', 'contactPrefix', 'contactPhone', 'country', 'state']] # Name the columns

In [13]:
df

,contributorName,payeeName,recipientName,amount,date,name,url,filerType,organizationName,lobbyistPrefix,...,noContributions,pacs,contributions,lobbyistID,address2,type,contactPrefix,contactPhone,country,state
0,Self,Colleen Holcomb for Delegate,Colleen Holcomb for Delegate,200.00,09/17/2019,NaN,https://disclosurespreview.house.gov/lc/lcxmlr...,O,Ms. Shari Rendall,,...,false,,,,,FECA,NaN,NaN,NaN,NaN
1,"Contura Energy, Inc. Political Action Committee",Healthcare Freedom Fund,Congressman Phil Roe,2500.00,07/05/2019,"Contura Energy, Inc. Political Action Committee",https://disclosurespreview.house.gov/lc/lcxmlr...,O,"CONTURA ENERGY, INC.",,...,false,,,,NaN,FECA,NaN,NaN,NaN,NaN
2,"Contura Energy, Inc. Political Action Committee",Volunteers for Shimkus,Congressman John Shimkus,1000.00,07/08/2019,"Contura Energy, Inc. Political Action Committee",https://disclosurespreview.house.gov/lc/lcxmlr...,O,"CONTURA ENERGY, INC.",,...,false,,,,NaN,FECA,NaN,NaN,NaN,NaN
3,"Contura Energy, Inc. Political Action Committee",Guy for Congress,Congressman Guy Reschenthaler,2500.00,07/11/2019,"Contura Energy, Inc. Political Action Committee",https://disclosurespreview.house.gov/lc/lcxmlr...,O,"CONTURA ENERGY, INC.",,...,false,,,,NaN,FECA,NaN,NaN,NaN,NaN
4,"Contura Energy, Inc. Political Action Committee",Fightin' Ninth PAC,Congressman Morgan Griffith,2500.00,09/23/2019,"Contura Energy, Inc. Political Action Committee",https://disclosurespreview.house.gov/lc/lcxmlr...,O,"CONTURA ENERGY, INC.",,...,false,,,,NaN,FECA,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72427,Self,John Carter for Congress,John Carter,250.00,09/11/2019,NaN,https://disclosurespreview.house.gov/lc/lcxmlr...,L,"BROWNSTEIN HYATT FARBER SCHRECK, LLP",Mr.,...,false,,,NaN,NaN,FECA,,NaN,NaN,NaN
72428,Self,Church for Wisconsin,Jason Church,100.00,11/11/2019,NaN,https://disclosurespreview.house.gov/lc/lcxmlr...,L,"BROWNSTEIN HYATT FARBER SCHRECK, LLP",Mr.,...,false,,,NaN,NaN,FECA,,NaN,NaN,NaN
72429,Self,The Guardian Fund PAC,Barry Loudermilk,250.00,11/12/2019,NaN,https://disclosurespreview.house.gov/lc/lcxmlr...,L,"BROWNSTEIN HYATT FARBER SCHRECK, LLP",Mr.,...,false,,,NaN,NaN,FECA,,NaN,NaN,NaN
72430,Self,Collins for Senator,Susan Collins,250.00,12/05/2019,NaN,https://disclosurespreview.house.gov/lc/lcxmlr...,L,"BROWNSTEIN HYATT FARBER SCHRECK, LLP",Mr.,...,false,,,NaN,NaN,FECA,,NaN,NaN,NaN


In [3]:
df.to_csv('data/processed/2019_YearEnd_XML.csv', sep=',', encoding='utf-8') # Export the whole thing to a CSV